# Bibliotecas

In [ ]:
import ranking
import salvar_resultados as sr

import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import funcoes_bert as fb

import time

## Constantes

In [ ]:
SEED = 42

# Lendo os Arquivos

In [ ]:
#arquivos = ['ale_1_1']
#arquivos = ['ale_5_1']
#arquivos = ['hn_1_1']
arquivos = ['hn_5_1']

lista_df_treino = []
lista_df_val = []
lista_df_teste = []
for arquivo in arquivos:

    df = pd.read_csv(f"Dados/Datasets/Treino-Validação/Treino/{arquivo}_treino.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df_treino.append(df)

    df = pd.read_csv(f"Dados/Datasets/Treino-Validação/Validação/{arquivo}_validação.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df_val.append(df)

    df = pd.read_csv(f"Dados/Datasets/Teste/{arquivo}_teste.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df_teste.append(df)

In [ ]:
#lista_df_val[0].dtypes

In [ ]:
#print(f"Tamanho do Dataset de Treino 1 por 1:\n\n\t| Ale\t| Hard\t|\n1 por 1\t| {lista_df_treino[0].shape[0]}\t| {lista_df_treino[2].shape[0]}\t|\n5 por 1\t| {lista_df_treino[1].shape[0]}\t| {lista_df_treino[3].shape[0]}\t|")
#print(f"\nTamanho do Dataset de Validação 1 por 1:\n\n\t| Ale\t| Hard\t|\n1 por 1\t| {lista_df_val[0].shape[0]}\t| {lista_df_val[2].shape[0]}\t|\n5 por 1\t| {lista_df_val[1].shape[0]}\t| {lista_df_val[3].shape[0]}\t|")
#print(f"\nTamanho do Dataset de Teste 1 por 1:\n\n\t| Ale\t| Hard\t|\n1 por 1\t| {lista_df_teste[0].shape[0]}\t| {lista_df_teste[2].shape[0]}\t|\n5 por 1\t| {lista_df_teste[1].shape[0]}\t| {lista_df_teste[3].shape[0]}\t|")


In [ ]:
from transformers import TFBertForSequenceClassification

# BERT

In [ ]:
def salvar_modelo_bert(modelo, nome):
    
    nome_arquivo = f"Dados/Modelos/BERT/{nome}/"
    modelo.save_pretrained(nome_arquivo)

In [ ]:
def criar_dicionario(indice, titulo, ean, categoria, colunas):
    
    return {
            colunas[0] : indice, colunas[1] : titulo, colunas[2] : ean, colunas[3] : categoria
           }


def criar_df_match(df_concat, ean_repetido, coluna_saida = 'titulo_sa'):

    COLUNAS = ("indice", coluna_saida, "ean", "categoria")

    df_matches = pd.DataFrame(columns = COLUNAS)
    for ean in ean_repetido:

        # pega o indice da primeira linha com aquele EAN
        filtro = (df_concat['ean'] == ean)
        indice = next(iter(filtro.index[filtro]))

        dicionario = criar_dicionario(
                                    indice = indice,
                                    titulo = df_concat.loc[indice][coluna_saida],
                                    ean = df_concat.loc[indice]['ean'],
                                    categoria = df_concat.loc[indice]['categoria'],
                                    colunas = COLUNAS
                                    )

        df_matches = df_matches.append(dicionario, ignore_index = True)

    df_matches.sort_values('indice', inplace = True)
    df_matches.reset_index(drop = True, inplace = True)

    return df_matches


## Rodando o BERT

In [ ]:
dir_metricas = "Dados/Resultados/Ranqueado/BERT"

In [ ]:
lista_df_resultado = []
#tam = 51
for nome, df_treino, df_val, df_teste in zip(arquivos, lista_df_treino, lista_df_val, lista_df_teste):

    '''df_teste = df_teste[:tam]
    df_treino = df_teste[:tam]
    df_val = df_teste[:tam]'''
    
    df_concat, df_matches = ranking.criar_df_teste(df_teste, "titulo_1", "titulo_2", 'titulo')

    X_treino = df_treino[["titulo_1", "titulo_2"]]
    X_val = df_val[["titulo_1", "titulo_2"]]
    X_teste = df_concat["titulo"]

    y_treino = df_treino["match"].to_list()
    y_val = df_val["match"].to_list()
    #y_teste = df_teste["match"].to_list()

    inicio_tempo = time.time()

    #(modelo, historico, embedding) = fb.pipeline_bert(X_treino, y_treino, X_val, y_val, X_teste)
    embedding = fb.pipeline_bert(X_teste)
    resultado = ranking.calcular_dis_2_vetores_cond(embedding, df_matches)
    
    final_tempo = time.time()
    
    # adicionando o tempo que demorou para o algoritmo rodar
    ranking.calcular_tempo(df_matches, inicio_tempo, final_tempo)

    # calcular acuracia_k e match rank
    indices, valores = ranking.calcular_metricas(df_matches, df_concat, resultado)

    #pd.DataFrame.from_dict(historico.history).to_csv(f'Dados/Resultados/Ranqueado/BERT/Histórico/{nome}_historico.csv', index = False)

    sr.salvar_resultado(nome, df_matches, dir_metricas)
    sr.salvar_resultado_categoria(nome, df_matches, dir_metricas)

    #salvar_modelo_bert(modelo, nome)

    break

In [ ]:
    a

# Resultados

In [21]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']

lista_df_resultado = []
for arquivo in arquivos:

    df_r = pd.read_csv(f"Dados/Resultados/Ranqueado/BERT/{arquivo}_métricas.csv")
    lista_df_resultado.append(df_r)

ale_1_1

In [22]:
lista_df_resultado[0][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,548.000000,548.000000,548.000000,548.000000,548.000000
mean,0.540146,0.666058,0.762774,85.678832,0.579572
std,0.498841,0.472050,0.425771,238.998067,0.464621
min,0.000000,0.000000,0.000000,1.000000,0.000430
25%,0.000000,0.000000,1.000000,1.000000,0.023124
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,43.250000,1.000000
max,1.000000,1.000000,1.000000,2323.000000,1.000000


ale_5_1

In [23]:
lista_df_resultado[1][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,1658.000000,1658.000000,1658.000000,1658.000000,1658.000000
mean,0.905911,0.928227,0.951146,48.396261,0.913138
std,0.292041,0.258190,0.215628,355.588530,0.272456
min,0.000000,0.000000,0.000000,1.000000,0.000139
25%,1.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,7202.000000,1.000000


hn_1_1

In [24]:
lista_df_resultado[2][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,584.000000,584.000000,584.000000,584.000000,584.000000
mean,0.590753,0.686644,0.792808,82.027397,0.626076
std,0.492116,0.464255,0.405642,259.091584,0.457989
min,0.000000,0.000000,0.000000,1.000000,0.000419
25%,0.000000,0.000000,1.000000,1.000000,0.040000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,25.000000,1.000000
max,1.000000,1.000000,1.000000,2386.000000,1.000000


hn_5_1

In [25]:
lista_df_resultado[3][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,1181.000000,1181.000000,1181.000000,1181.000000,1181.000000
mean,0.948349,0.959356,0.964437,35.070279,0.951394
std,0.221415,0.197547,0.185277,314.552452,0.209906
min,0.000000,0.000000,0.000000,1.000000,0.000153
25%,1.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,6556.000000,1.000000
